In [ ]:
import os
import numpy as np
import xml.etree.ElementTree as ET
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import VGG16  # Changed from ResNet50
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.regularizers import l2

In [ ]:
batch_size = 32

input_shape = (224, 224, 3)

In [ ]:
dataset_path = '/content/drive/MyDrive/oocyte.v16i.voc'


In [ ]:
train_path = os.path.join(dataset_path, 'Train')
test_path = os.path.join(dataset_path, 'Test')
valid_path = os.path.join(dataset_path, 'Valid')

In [ ]:
def load_images_and_labels(image_dir, label_dir):
    images = []
    labels = []

    # List all XML files in the label directory
    xml_files = [filename for filename in os.listdir(label_dir) if filename.endswith('.xml')]
    print("Found XML Files:", xml_files)

    # Iterate over image files
    for image_filename in os.listdir(image_dir):
        if image_filename.endswith('.jpg'):
            image_path = os.path.join(image_dir, image_filename)
            label_filename = os.path.splitext(image_filename)[0] + '.xml'

            label_path = os.path.join(label_dir, label_filename)
            print("Checking Path:", label_path)  # Print the label path for troubleshooting

            # Load image
            image = load_img(image_path, target_size=input_shape[:2])
            image = img_to_array(image)
            images.append(image)

            if os.path.exists(label_path):
                # Parse XML label
                tree = ET.parse(label_path)
                root = tree.getroot()
                label_element = root.find('object/name')  # Find the <name> element under <object>
                if label_element is not None:
                    label = label_element.text.strip()  # Extract the label text and remove whitespace
                    labels.append(label)
                else:
                    labels.append("unknown")  # Placeholder label for images without a label
            else:
                labels.append("unknown")  # Placeholder label for images without a label

    return np.array(images), np.array(labels)

In [ ]:
train_images, train_labels = load_images_and_labels(train_path, train_path)
test_images, test_labels = load_images_and_labels(test_path, test_path)
valid_images, valid_labels = load_images_and_labels(valid_path, valid_path)

In [ ]:
label_encoder = LabelEncoder()


In [ ]:
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)
valid_labels_encoded = label_encoder.transform(valid_labels)

In [ ]:
num_classes = len(label_encoder.classes_)
train_labels_categorical = to_categorical(train_labels_encoded, num_classes)
test_labels_categorical = to_categorical(test_labels_encoded, num_classes)
valid_labels_categorical = to_categorical(valid_labels_encoded, num_classes)

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)  # Changed from ResNet50


58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)  # Adjusted units and added dropout
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(train_images, train_labels_categorical, epochs=10, validation_data=(valid_images, valid_labels_categorical))


Epoch 1/10
12/12 [==============================] - 562s 47s/step - loss: 33.9341 - accuracy: 0.5184 - val_loss: 0.8244 - val_accuracy: 0.2078
Epoch 2/10
12/12 [==============================] - 561s 47s/step - loss: 0.7315 - accuracy: 0.6374 - val_loss: 0.8952 - val_accuracy: 0.7662
Epoch 3/10
12/12 [==============================] - 549s 46s/step - loss: 0.7564 - accuracy: 0.6742 - val_loss: 0.6396 - val_accuracy: 0.7662
Epoch 4/10
12/12 [==============================] - 544s 45s/step - loss: 0.7185 - accuracy: 0.6856 - val_loss: 0.7129 - val_accuracy: 0.7662
Epoch 5/10
12/12 [==============================] - 550s 46s/step - loss: 0.7085 - accuracy: 0.6884 - val_loss: 0.5988 - val_accuracy: 0.7662
Epoch 6/10
12/12 [==============================] - 567s 47s/step - loss: 0.6647 - accuracy: 0.6969 - val_loss: 0.5943 - val_accuracy: 0.7662
Epoch 7/10
12/12 [==============================] - 561s 47s/step - loss: 0.6783 - accuracy: 0.6714 - val_loss: 0.6350 - val_accuracy: 0.7662
Epoch

In [ ]:
train_acc = history.history['accuracy'][-1]
print("Training Accuracy:", train_acc)

val_acc = history.history['val_accuracy'][-1]
print("Validation Accuracy:", val_acc)

test_loss, test_acc = model.evaluate(test_images, test_labels_categorical)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

Training Accuracy: 0.6827195286750793
Validation Accuracy: 0.7662337422370911
3/3 [==============================] - 31s 9s/step - loss: 0.6274 - accuracy: 0.7067
Test Loss: 0.6273614764213562
Test Accuracy: 0.7066666483879089
